# SentEval on Local

SentEval is a widely used benchmarking tool for evaluating general-purpose sentence embeddings. It provides a simple interface for evaluating your embeddings on up to 17 supported downstream tasks (such as sentiment classification, natural language inference, semantic similarity, etc.)

Running SentEval locally is simple. Clone the [repository](https://github.com/facebookresearch/SentEval), follow their setup instructions to get the data for the transfer tasks, and implement two functions `prepare(params, samples)` and `batcher(params, batch)` specific to your model. The authors provide some guidance on how to do this in the [examples](https://github.com/facebookresearch/SentEval/tree/master/examples) directory of their repository. In this notebook we show an example for evaluating the GenSen model on the available STS downstream tasks.

### 00 Global Settings

In [14]:
import os
import sys
import json
import torch
import pandas as pd
import scrapbook as sb

sys.path.append("../../")
from utils_nlp.models.gensen.gensen import GenSen, GenSenSingle
from utils_nlp.eval.senteval import SentEvalRunner
from utils_nlp.dataset.url_utils import maybe_download

print("System version: {}".format(sys.version))
print("PyTorch version: {}".format(torch.__version__))

System version: 3.6.8 |Anaconda, Inc.| (default, Dec 30 2018, 01:22:34) 
[GCC 7.3.0]
PyTorch version: 1.0.1


In [6]:
sent_path = TemporaryDirectory()
gensen_path = TemporaryDirectory()
PATH_TO_SENTEVAL = sent_path.name
PATH_TO_GENSEN = gensen_path.name

In [2]:
PATH_TO_SENTEVAL = (
    "../../../SentEval"
)  # Set this path to where you have cloned the senteval source code
PATH_TO_GENSEN = (
    "../../../gensen"
)  # Set this path to where you have cloned the gensen source code

In [8]:
print(PATH_TO_SENTEVAL)
print(PATH_TO_GENSEN)

/tmp/tmpjd7js8_t
/tmp/tmpc0by1eg4


In [25]:
nli_large_bothskip_vocab = "https://genseniclr2018.blob.core.windows.net/models/nli_large_bothskip_vocab.pkl"
maybe_download(url=nli_large_bothskip_vocab, work_directory=PATH_TO_GENSEN)
nli_large_bothskip = "https://genseniclr2018.blob.core.windows.net/models/nli_large_bothskip.model"
maybe_download(url=nli_large_bothskip, work_directory=PATH_TO_GENSEN)


100%|██████████| 197k/197k [00:11<00:00, 16.8kKB/s] 


'/tmp/tmpc0by1eg4/nli_large_bothskip.model'

### 01 SentEval Settings

In [21]:
transfer_tasks = ["STSBenchmark", "STS12", "STS13", "STS14", "STS15", "STS16"]
params_senteval = {
    "usepytorch": True,
    "kfold": 10,
}

### 02 GenSen Settings

In [22]:
model_params = {}
#model_params["folder_path"] = os.path.join(PATH_TO_GENSEN, "data/models")
model_params["folder_path"] = PATH_TO_GENSEN
model_params["prefix"] = "nli_large_bothskip"
model_params["pretrain"] = os.path.join(
    PATH_TO_GENSEN, "data/embedding/glove.840B.300d.h5"
)
model_params["cuda"] = torch.cuda.is_available()

print("model params: {}".format(json.dumps(model_params, indent=4)))

model params: {
    "folder_path": "/tmp/tmpc0by1eg4",
    "prefix": "nli_large_bothskip",
    "pretrain": "/tmp/tmpc0by1eg4/data/embedding/glove.840B.300d.h5",
    "cuda": true
}


### 03 SentEval Functions

As specified in the SentEval [repo](https://github.com/facebookresearch/SentEval#how-to-use-senteval), we implement 2 functions:

<b>prepare</b> (sees the whole dataset of each task and can thus construct the word vocabulary, the dictionary of word vectors etc)         
<b>batcher</b> (transforms a batch of text sentences into sentence embeddings)

In [23]:
def prepare(params, samples):
    vocab = set()
    for sample in samples:
        if params.current_task != "TREC":
            sample = " ".join(sample).lower().split()
        else:
            sample = " ".join(sample).split()
        for word in sample:
            if word not in vocab:
                vocab.add(word)

    vocab.add("<s>")
    vocab.add("<pad>")
    vocab.add("<unk>")
    vocab.add("</s>")
    # Optional vocab expansion
    # params["model"].vocab_expansion(vocab)


def batcher(params, batch):
    # batch contains list of words
    max_tasks = ["MR", "CR", "SUBJ", "MPQA", "ImageCaptionRetrieval"]
    if params.current_task in max_tasks:
        strategy = "max"
    else:
        strategy = "last"

    sentences = [" ".join(s).lower() for s in batch]
    _, embeddings = params["model"].get_representation(
        sentences, pool=strategy, return_numpy=True
    )
    return embeddings

### 04 Run SentEval on GenSen

In [26]:
gensen = GenSenSingle(
    model_folder=model_params["folder_path"],
    filename_prefix=model_params["prefix"],
    pretrained_emb=model_params["pretrain"],
    cuda=model_params["cuda"],
)

ser = SentEvalRunner(path_to_senteval=PATH_TO_SENTEVAL)
ser.set_transfer_data_path("data")
ser.set_transfer_tasks(transfer_tasks)
ser.set_model(gensen)
ser.set_params(params_senteval)
results = ser.run(batcher, prepare)

FileNotFoundError: [Errno 2] No such file or directory: '/tmp/tmpjd7js8_t/data/downstream/STS/STSBenchmark/sts-train.csv'

Print selected metrics from the model's results on the transfer tasks as a table.

In [7]:
eval_metrics = ser.log_mean(results, selected_metrics=["pearson", "spearman"])
eval_metrics.head(eval_metrics.shape[0])

,pearson,spearman
STSBenchmark,0.778,0.780
STS12,0.598,0.603
STS13,0.542,0.556
STS14,0.639,0.625
STS15,0.727,0.729
STS16,0.659,0.663


In [8]:
# Persist output to scrapbook
sb.glue("results", eval_metrics.to_dict())